In [5]:
%pip install -qq -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
from minio import Minio
import pandas as pd

In [7]:
minio_client = Minio(
    "minio-api.digitalhub-test.smartcommunitylab.it",
    access_key="AK",
    secret_key="SK"
)

# Download overtourism datasets from the minio bucket

In [9]:
objects = minio_client.list_objects("datalake", prefix="projects/tourism", recursive=True)
for obj in objects:
    minio_client.fget_object(obj.bucket_name, obj.object_name, f"datasets/{obj.object_name[len('projects/tourism/'):]}")